# Train a deep learning model
In this notebook you will train a deep learning model to classify the descriptions of car components as compliant or non-compliant. 

Each document in the supplied training data set is a short text description of the component as documented by an authorized technician. 
The contents include:
- Manufacture year of the component (e.g. 1985, 2010)
- Condition of the component (poor, fair, good, new)
- Materials used in the component (plastic, carbon fiber, steel, iron)

The compliance regulations dictate:
*Any component manufactured before 1995 or in fair or poor condition or made with plastic or iron is out of compliance.*

For example:
* Manufactured in 1985 made of steel in fair condition -> **Non-compliant**
* Good condition carbon fiber component manufactured in 2010 -> **Compliant**
* Steel component manufactured in 1995 in fair condition -> **Non-Compliant**

The labels present in this data are 0 for compliant, 1 for non-compliant.

The challenge with classifying text data is that deep learning models only undertand vectors (e.g., arrays of numbers) and not text. To encode the car component descriptions as vectors, we use an algorithm from Stanford called [GloVe (Global Vectors for Word Representation)](https://nlp.stanford.edu/projects/glove/). GloVe provides us pre-trained vectors that we can use to convert a string of text into a vector. 

## Setup
To begin, you will need to provide the following information about your Azure Subscription. 

In the following cell, be sure to set the values for `subscription_id`, `resource_group`, `workspace_name` and `workspace_region` as directed by the comments (*these values can be acquired from the Azure Portal*). Also provide the values for the pre-created AKS cluster (`aks_resource_group_name` and `aks_cluster_name`). 

If you are in doubt of any of these values, do the following:
1. Navigate to the Azure Portal and login with the credentials provided.
2. From the left hand menu, under Favorites, select `Resource Groups`.
3. In the list, select the resource group with the name similar to `tech-immersion-onnx-XXXXX`.
4. From the Overview tab, capture the desired values.

Execute the following cell by selecting the `>|Run` button in the command bar above.


In [ ]:
#Provide the Subscription ID of your existing Azure subscription
subscription_id = "" # <- needs to be the subscription with the ONNX resource group

#Provide values for the existing "ONNX" Resource Group 
resource_group = "tech-immersion-onnx-xxxxx" # <- replace xxxxx with the values from your ONNX resource group name

#Provide the Workspace Name and Azure Region of the Azure Machine Learning Workspace
workspace_name = "gpu-tech-immersion-aml-xxxxx" # <- replace xxxxx with the values from your ONNX resource group name
workspace_region = "eastus2" # <- region of your ONNX resource group (other options include eastus, westcentralus, southeastasia, australiaeast, westeurope)

In [ ]:
# constants, you can leave these values as they are or experiment with changing them after you have completed the notebook once
experiment_name = 'deep-learning'
project_folder = './dl'
deployment_folder = './deploy'
onnx_export_folder = './onnx'

# this is the URL to the CSV file containing the GloVe vectors
glove_url = ('https://quickstartsws9073123377.blob.core.windows.net/'
             'azureml-blobstore-0d1c4218-a5f9-418b-bf55-902b65277b85/'
             'quickstarts/connected-car-data/glove.6B.100d.txt')

# this is the URL to the CSV file containing the care component descriptions
data_url = ('https://quickstartsws9073123377.blob.core.windows.net/'
            'azureml-blobstore-0d1c4218-a5f9-418b-bf55-902b65277b85/'
            'quickstarts/connected-car-data/connected-car_components.csv')

# this is the name of the AML Compute cluster
cluster_name = "gpucluster"

# this is the name of the web service deployed on the AKS cluster
aks_service_name ='contoso-service'

# Create the Azure Machine Learning resources

The Azure Machine Learning SDK provides a comprehensive set of a capabilities that you can use directly within a notebook including:
- Creating a **Workspace** that acts as the root object to organize all artifacts and resources used by Azure Machine Learning.
- Creating **Experiments** in your Workspace that capture versions of the trained model along with any desired model performance telemetry. Each time you train a model and evaluate its results, you can capture that run (model and telemetry) within an Experiment.
- Creating **Compute** resources that can be used to scale out model training, so that while your notebook may be running in a lightweight container in Azure Notebooks, your model training can actually occur on a powerful cluster that can provide large amounts of memory, CPU or GPU. 
- Using **Automated Machine Learning (AutoML)** to automatically train multiple versions of a model using a mix of different ways to prepare the data and different algorithms and hyperparameters (algorithm settings) in search of the model that performs best according to a performance metric that you specify. 
- Packaging a Docker **Image** that contains everything your trained model needs for scoring (prediction) in order to run as a web service.
- Deploying your Image to either Azure Kubernetes or Azure Container Instances, effectively hosting the **Web Service**.

In Azure Notebooks, all of the libraries needed for Azure Machine Learning are pre-installed. To use them, you just need to import them. Run the following cell to do so:

In [ ]:
import logging
import os
import random
import re

from matplotlib import pyplot as plt
from matplotlib.pyplot import imshow
import numpy as np
import pandas as pd

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.core.compute import AksCompute, ComputeTarget
from azureml.core.webservice import Webservice, AksWebservice
from azureml.core.image import Image
from azureml.core.model import Model
from azureml.train.automl import AutoMLConfig
from azureml.train.automl.run import AutoMLRun
from azureml.core import Workspace

## Create and connect to an Azure Machine Learning Workspace
Run the following cell to create a new Azure Machine Learning **Workspace** and save the configuration to disk (next to the Jupyter notebook). 

**Important Note**: You will be prompted to login in the text that is output below the cell. Be sure to navigate to the URL displayed and enter the code that is provided. Once you have entered the code, return to this notebook and wait for the output to read `Workspace configuration succeeded`.

In [ ]:
# By using the exist_ok param, if the worskpace already exists you get a reference to the existing workspace
# allowing you to re-run this cell multiple times as desired (which is fairly common in notebooks).
ws = Workspace.create(
    name = workspace_name,
    subscription_id = subscription_id,
    resource_group = resource_group, 
    location = workspace_region,
    exist_ok = True)

ws.write_config()
print('Workspace configuration succeeded')


### Create AML Compute Cluster
Now you are ready to create the GPU compute cluster. Run the following cell to create a new compute cluster (or retrieve the existing cluster if it already exists). The code below will create a *GPU based* cluster where each node in the cluster is of the size `Standard_NC6s`, and the cluster will start with at 1 such node and can scale out to 2. 

In [ ]:
### Create AML GPU based Compute Cluster
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target.')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_NC12',
                                                           min_nodes=1, max_nodes=1)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    compute_target.wait_for_completion(show_output=True)

# Use the 'status' property to get a detailed status for the current AmlCompute. 
print(compute_target.status.serialize())

### Create a TensorFlow estimator

An estimator allows you to define the job that will execute when you run an experiment. The `TensorFlow` estimator is specially configured to handle executing runs that use TensorFlow, such as the Keras training script you will create shortly. Run the following cell to create the estimator.

In [ ]:
from azureml.train.dnn import TensorFlow

keras_est = TensorFlow(source_directory=project_folder,
                       compute_target=compute_target,
                       entry_script='train.py',
                       conda_packages=['pandas'],
                       pip_packages=['keras==2.2.4'], # just add keras through pip
                       use_gpu=True)

## Remotely train a deep learning model using the Azure ML Compute
In the following cells, you will *not* train the model against the data you just downloaded using the resources provided by Azure Notebooks. Instead, you will deploy an Azure ML Compute cluster that will download the data and use a trainings script to train the model. In other words, all of the training will be performed remotely with respect to this notebook. 


In [ ]:
# create project folder
if not os.path.exists(project_folder):
    os.makedirs(project_folder)

### Create the training script

In [ ]:
%%writefile $project_folder/train.py

import os
import numpy as np
import pandas as pd

import keras
from keras import models 
from keras import layers
from keras import optimizers

def get_data():
    data_url = "https://databricksdemostore.blob.core.windows.net/data/connected-car/connected-car_components.csv"
    car_components_df = pd.read_csv(data_url)
    components = car_components_df["text"].tolist()
    labels = car_components_df["label"].tolist()
    return { "components" : components, "labels" : labels }

def download_glove():
    print("Downloading GloVe embeddings...")
    import urllib.request
    urllib.request.urlretrieve('https://databricksdemostore.blob.core.windows.net/data/connected-car/glove.6B.100d.txt', 'glove.6B.100d.txt')
    print("Download complete.")

download_glove()

# Load the car components labeled data
print("Loading car components data...")
data_url = "https://databricksdemostore.blob.core.windows.net/data/connected-car/connected-car_components.csv"
car_components_df = pd.read_csv(data_url)
components = car_components_df["text"].tolist()
labels = car_components_df["label"].tolist()
print("Loading car components data completed.")

# split data 60% for trianing, 20% for validation, 20% for test
print("Splitting data...")
train, validate, test = np.split(car_components_df.sample(frac=1), [int(.6*len(car_components_df)), int(.8*len(car_components_df))])
print(train.shape)
print(test.shape)
print(validate.shape)

# use the Tokenizer from Keras to "learn" a vocabulary from the entire car components text
print("Tokenizing data...")
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 100                                           
training_samples = 90000                                 
validation_samples = 5000    
max_words = 10000      

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(components)
sequences = tokenizer.texts_to_sequences(components)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

indices = np.arange(data.shape[0])                     
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_train = data[:training_samples]
y_train = labels[:training_samples]

x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

x_test = data[training_samples + validation_samples:]
y_test = labels[training_samples + validation_samples:]
print("Tokenizing data complete.")

# apply the vectors provided by GloVe to create a word embedding matrix
print("Applying GloVe vectors...")
glove_dir =  './'

embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

embedding_dim = 100

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector    
print("Applying GloVe vectors compelted.")

# use Keras to define the structure of the deep neural network   
print("Creating model structure...")
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

# fix the weights for the first layer to those provided by the embedding matrix
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False
print("Creating model structure completed.")

print("Training model...")
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=10,
                    batch_size=32,
                    validation_data=(x_val, y_val))
print("Training model completed.")

print("Saving model files...")
# create a ./outputs/model folder in the compute target
# files saved in the "./outputs" folder are automatically uploaded into run history
os.makedirs('./outputs/model', exist_ok=True)

# serialize NN architecture to JSON
model_json = model.to_json()
# save model JSON
with open('./outputs/model/model.json', 'w') as f:
    f.write(model_json)
# save model weights
model.save_weights('./outputs/model/model.h5')
print("model saved in ./outputs/model folder")
print("Saving model files completed.")


## Submit the training run

The code pattern to submit a training run to Azure Machine Learning compute targets is always:

- Create an experiment to run.
- Submit the experiment.
- Wait for the run to complete.

### Create the experiment

In [ ]:
experiment = Experiment(ws, experiment_name)

### Submit the experiment

In [ ]:
run = experiment.submit(keras_est)

Wait for the run to complete by executing the following cell. Note that this process will perform the following:
- Build and deploy the container to Azure Machine Learning compute (~8 minutes)
- Execute the training script (~2 minutes)

If you change only the training script and re-submit, it will run faster the second time because the necessary container is already prepared so the time requried is just that for executing the training script.

In [ ]:
# if you want to see detailed output of the deployment and training run, uncomment the following.
# note that this will add about 10 minutes to the time it takes to complete the lab as you will be blocked from executing any other cells until this cell finishes.
run.wait_for_completion(show_output = True)

# Understanding the training script
The training script shown previously does a lot, lets break it apart into smaller parts that you can run locally. 

## Download the GloVe embeddings to your environment.
Run the following cell to download the embeddings to the `data` folder in your environment. Note: this may take a **few minutes** as the GloVe file is about 340 MB. 

In [ ]:
import urllib.request
import os
os.makedirs('data', exist_ok=True)
urllib.request.urlretrieve(glove_url, './data/glove.6B.100d.txt')

Run the following cells to import the needed libraries.

In [ ]:
import numpy as np
import pandas as pd

import keras
from keras import models 
from keras import layers
from keras import optimizers

Now that you have downloaded the data, load it into a Pandas DataFrame by running the following cell.

In [ ]:
# Load the car components labeled data
car_components_df = pd.read_csv(data_url)
components = car_components_df["text"].tolist()
labels = car_components_df["label"].tolist()

In [ ]:
# split data 60% for training, 20% for validation, 20% for test
train, validate, test = np.split(car_components_df.sample(frac=1), [int(.6*len(car_components_df)), int(.8*len(car_components_df))])
print(train.shape)
print(test.shape)
print(validate.shape)

In the following cell, you use the Tokenizer from Keras to "learn" a vocabulary from the entire car components text. Then the data (both the text and the compliance labels) is split into three subsets, one that will be used for training the deep learning model, one that will be used during training batches to tune the model weights and one that will be used after the model is trained to evaluate how it performs on data the model has never seen. 

Run the following cell.

In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 100                                           
training_samples = 90000                                 
validation_samples = 5000    
max_words = 10000      

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(components)
sequences = tokenizer.texts_to_sequences(components)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(labels)
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

indices = np.arange(data.shape[0])                     
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_train = data[:training_samples]
y_train = labels[:training_samples]

x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

x_test = data[training_samples + validation_samples:]
y_test = labels[training_samples + validation_samples:]


Now take a look at how the text was encoded as an array in the above. Run the following cell to take a peek.

In [ ]:
print("The text '{text}' is represented as the vector '{data}'".format(text=components[indices[0]], data=x_train[0]))

Next, you will apply the vectors provided by GloVe to create a word embedding matrix. This matrix will be used shortly to set the model wights of the first layer of the deep neural network. 

Run the following cell.

In [ ]:
glove_dir =  './data'

embeddings_index = {}
f = open(os.path.join(glove_dir, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

embedding_dim = 100

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    if i < max_words:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector    

In the next cell, you will use Keras to define the structure of the deep neural network. The network graph you build in this case has four layers. 

Run the following cell to structure the network and view a summary description of it.

In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

Rather than train model from scratch on this car components text, you can instead use the embedding matrix derived from GloVe. In effect this boosts the model's understanding of text, because the GloVe vectors was trained against a large corpus of text in Wikipedia. 

Run the following cell to fix the weights for the first layer to those provided by the embedding matrix.

In [ ]:
model.layers[0].set_weights([embedding_matrix])
model.layers[0].trainable = False

Now you are ready to train the model.

Run the following cell to train the model. This will take **3-4 minutes** as you are using CPU instead of GPU enabled cluster.

In [ ]:
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
history = model.fit(x_train, y_train,
                    epochs=3,
                    batch_size=32,
                    validation_data=(x_val, y_val))
model.save_weights(os.path.join(glove_dir,'pre_trained_glove_model.h5'))

Take a look to see how the model training went. If curves for training accuracy and validation accuracy come together, you are in good shape!

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

fig = plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt

You can also evaluate how accurately the model performs against data it has not seen. Run the following cell to see the accuracy (it is the second number in the array displayed, on a scale from 0 to 1).

In [ ]:
model.load_weights(os.path.join(glove_dir,'pre_trained_glove_model.h5'))
model.evaluate(x_test, y_test)

At this point you have explored how the model is trained and used locally. Let's return to the model you had previously trained remotely using the Azure Machine Learning compute.

## Download the model files from the run

In the training script, the Keras model is saved into two files, model.json and model.h5, in the outputs/models folder on the GPU cluster AmlCompute node. Azure ML automatically uploaded anything written in the ./outputs folder into run history file store. Subsequently, we can use the run object to download the model files. They are under the the outputs/model folder in the run history file store, and are downloaded into a local folder named model.

In [ ]:
# create a model folder in the current directory
os.makedirs('./model', exist_ok=True)

for f in run.get_file_names():
    if f.startswith('outputs/model'):
        output_file_path = os.path.join('./model', f.split('/')[-1])
        print('Downloading from {} to {} ...'.format(f, output_file_path))
        run.download_file(name=f, output_file_path=output_file_path)

In [ ]:
from keras.models import model_from_json

embedding_dim = 100
maxlen = 100                                             
max_words = 10000    

from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.summary()

# load weights into new model
model.load_weights("model/model.h5")
print("Model loaded from disk.")

# Converting a Keras model to ONNX
In the steps that follow, you will convert Keras model you just trained to the ONNX format. This will enable you to use this model for classification in a very broad range of environments including:

- Web services 
- iOS and Android mobile apps
- Windows apps
- IoT devices

Converting a Keras model requires the use of the `onnxmltools`, `onnx` and `protobuf` libraries. These libraries come pre-installed in Azure Notebooks.

Instead of waiting for the training job to complete, we have provide you with a previously trained model you can quickly re-load to experiment with converting it to ONNX. Run the following cell to load this model:

In [ ]:
cwd = os.getcwd()
if cwd.endswith('/deploy'):
    os.chdir('../')

embedding_dim = 100
maxlen = 100                                             
max_words = 10000    

from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense

model = Sequential()
model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
model.add(Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# download the pretrained model from the deep learning quickstart
os.makedirs('./model', exist_ok=True)
model_url = ('https://quickstartsws9073123377.blob.core.windows.net/'
             'azureml-blobstore-0d1c4218-a5f9-418b-bf55-902b65277b85/'
             'quickstarts/dl-trained-model/model.h5')
urllib.request.urlretrieve(model_url, os.path.join('./model', 'model.h5'))

# load weights into new model
model.load_weights(os.path.join('./model', 'model.h5'))
print("Model loaded from disk.")
print(model.summary())

Convert the model to ONNX by running the following cell.

In [ ]:
# create a deployment folder in the current directory
os.makedirs(deployment_folder, exist_ok=True)

# create an onnx subfolder under deployment
os.makedirs(os.path.join(deployment_folder, onnx_export_folder), exist_ok=True)

import onnxmltools

# Convert the Keras model to ONNX
onnx_model_name = 'component_compliance.onnx'
converted_model = onnxmltools.convert_keras(model, onnx_model_name, target_opset=7)

# Save the model locally...
onnx_model_path = os.path.join(deployment_folder, onnx_export_folder)
os.makedirs(onnx_model_path, exist_ok=True)
#winmltools.save_model(converted_model, os.path.join(os.path.join(deployment_folder, onnx_export_folder), onnx_model_name))
onnxmltools.utils.save_model(converted_model, os.path.join(onnx_model_path,onnx_model_name))

The above cell created a new file called `component_compliance.onnx` that contains the ONNX version of the model. 

To be able to use an ONNX model for inferencing, your environment only needs to have the `onnxruntime` installed. Run the following cell to install the `onnxruntime`. 

In [ ]:
%%sh
pip install onnxruntime

Now try using this ONNX model to classify a component description by running the following cell. Remeber the prediction will be a value close to 0 (non-compliant) or to 1 (compliant).

In [ ]:
import onnxruntime

# Load the ONNX model
onnx_session = onnxruntime.InferenceSession(os.path.join(os.path.join(deployment_folder, onnx_export_folder), onnx_model_name))

# Grab one sample from the test data set
test_sample = np.reshape(x_test.astype(np.float32)[2], (1,100))

# Run an ONNX session to classify the sample.
classify_output = onnx_session.run(None, {onnx_session.get_inputs()[0].name:test_sample})[0] 

print("Your model predicted the class `{pred}`, and the actual class was `{actual}`".format(pred=classify_output[0][0], actual=y_test[0]))

# Deploy Deep Learning ONNX format model as a web service
To demonstrate one example of using the ONNX format model in a new environment, you will deploy the ONNX model to a webservice. On the web server, the only component required by the model is the ONNX Runtime, which is used to load the model and use it for scoring. Neither Keras nor TensorFlow are required on the web server.

In this case, you will use the Azure Machine Learning service SDK to programmatically create a Workspace, register your model, create a container image for the web service that uses it and deploy that image on to an Azure Container Instance.

Run the following cells to create some helper functions that you will use for deployment.

In [ ]:
import azureml
from azureml.core import Workspace
from azureml.core.model import Model

In [ ]:
def getOrCreateWorkspace(subscription_id, resource_group, workspace_name, workspace_region):
    # By using the exist_ok param, if the workspace already exists we get a reference to the existing workspace instead of an error
    ws = Workspace.create(
        name = workspace_name,
        subscription_id = subscription_id,
        resource_group = resource_group, 
        location = workspace_region,
        exist_ok = True)
    return ws

In [ ]:
def deployModelAsWebService(ws, model_folder_path="models", model_name="component_compliance", 
                scoring_script_filename="scoring_service.py", 
                conda_packages=['numpy','pandas'],
                pip_packages=['azureml-sdk','onnxruntime'],
                conda_file="dependencies.yml", runtime="python",
                cpu_cores=1, memory_gb=1, tags={'name':'scoring'},
                description='Compliance classification web service.',
                service_name = "complianceservice"
               ):
    # notice for the model_path, we supply the name of the outputs folder without a trailing slash
    # this will ensure both the model and the customestimators get uploaded.
    print("Registering and uploading model...")
    registered_model = Model.register(model_path=model_folder_path, 
                                      model_name=model_name, 
                                      workspace=ws)

    # create a Conda dependencies environment file
    print("Creating conda dependencies file locally...")
    from azureml.core.conda_dependencies import CondaDependencies 
    mycondaenv = CondaDependencies.create(conda_packages=conda_packages, pip_packages=pip_packages)
    with open(conda_file,"w") as f:
        f.write(mycondaenv.serialize_to_string())
        
    # create container image configuration
    print("Creating container image configuration...")
    from azureml.core.image import ContainerImage
    image_config = ContainerImage.image_configuration(execution_script = scoring_script_filename,
                                                      runtime = runtime,
                                                      conda_file = conda_file)
    
    # create ACI configuration
    print("Creating ACI configuration...")
    from azureml.core.webservice import AciWebservice, Webservice
    aci_config = AciWebservice.deploy_configuration(
        cpu_cores = cpu_cores, 
        memory_gb = memory_gb, 
        tags = tags, 
        description = description)

    # deploy the webservice to ACI
    print("Deploying webservice to ACI...")
    webservice = Webservice.deploy_from_model(
      workspace=ws, 
      name=service_name, 
      deployment_config=aci_config,
      models = [registered_model], 
      image_config=image_config
    )
    webservice.wait_for_deployment(show_output=True)
    
    return webservice

Your web service which knows how to load the model and use it for scoring needs saved out to a file for the Azure Machine Learning service SDK to deploy it. Run the following cell to create this file.

In [ ]:
%%writefile $deployment_folder/scoring_service.py
import sys
import os
import json
import numpy as np
import pandas as pd
from azureml.core.model import Model
import onnxruntime

def init():
    global model
    
    try:
        model_path = Model.get_model_path('component_compliance')
        model_file_path = os.path.join(model_path,'component_compliance.onnx')
        print('Loading model from:', model_file_path)
        
        # Load the ONNX model
        model = onnxruntime.InferenceSession(model_file_path)
    except Exception as e:
        print(e)
        
# note you can pass in multiple rows for scoring
def run(raw_data):
    try:
        print("Received input:", raw_data)
        
        input_data = np.array(json.loads(raw_data)).astype(np.float32)
        
        # Run an ONNX session to classify the input.
        result = model.run(None, {model.get_inputs()[0].name:input_data})[0] 
        result = result[0][0].item()
        
        # return just the classification index (0 or 1)
        return result
    except Exception as e:
        error = str(e)
        return error


Next, create your Workspace (or retrieve the existing one if it already exists) and deploy the model as a web service.

Before you run the cell, modify the value of the `service_name` parameter to replace `YOUR-UNIQUE-IDENTIFIER` so that it includes your identifier.

Run the cell to perform the deployment.

In [ ]:
# It is important to change the current working directory so that your generated scoring-service.py is at the root of it. 
# This is required by the Azure Machine Learning SDK
os.chdir(deployment_folder)

In [ ]:
ws =  getOrCreateWorkspace(subscription_id, resource_group, 
                   workspace_name, workspace_region)

print(os.getcwd())
webservice = deployModelAsWebService(ws, model_folder_path=onnx_export_folder, model_name="component_compliance", service_name = "complianceservice-zst")

Finally, test your deployed web service.

In [ ]:
import json

# choose a sample from the test data set to send
test_sample = np.reshape(x_test.astype(np.float32)[2], (1,100))
test_sample_json = json.dumps(test_sample.tolist())

# invoke the web service
result = webservice.run(input_data=test_sample_json)

result

You now have a working web service deployed that uses the ONNX version of your Keras deep learning model.